# Machine Learning for Seagrass Extent Prediction

In [2]:
from pystac.client import Client
from odc.stac import load
import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import joblib
from matplotlib import colors


from utils import scale, apply_masks, do_prediction, calculate_band_indices

## Loading Sentinel-2 GeoMAD

Load data, then create band indices and mask out areas we don't want to include.

In [3]:
catalog = "https://stac.digitalearthpacific.org"
client = Client.open(catalog)

In [4]:
# Ba Estuary
# bbox = [177.51971, -17.49416, 177.68452, -17.34430]
# datetime="2024"

In [5]:
# Komave
bbox = [177.72578562645654, -18.240565454997043, 177.8126842310554, -18.217513820319642]
datetime="2024"

In [6]:
items = client.search(
    collections=["dep_s2_geomad"], datetime=datetime, bbox=bbox
).item_collection()

print(f"Found {len(items)} items")

Found 1 items


In [7]:
data = load(
    items,
    bbox=bbox,
    measurements=[
        "nir",
        "red",
        "blue",
        "green",
        "emad",
        "smad",
        "bcmad",
        "count",
        "green",
        "nir08",
        "nir09",
        "swir16",
        "swir22",
        "coastal",
        "rededge1",
        "rededge2",
        "rededge3",
    ],
    chunks={"x": 2048, "y": 2048},
)

data

<xarray.Dataset> Size: 10MB
Dimensions:      (y: 269, x: 968, time: 1)
Coordinates:
  * y            (y) float64 2kB -2.05e+06 -2.05e+06 ... -2.052e+06 -2.052e+06
  * x            (x) float64 8kB 3.086e+06 3.086e+06 ... 3.096e+06 3.096e+06
    spatial_ref  int32 4B 3832
  * time         (time) datetime64[ns] 8B 2024-01-01
Data variables: (12/16)
    nir          (time, y, x) uint16 521kB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    red          (time, y, x) uint16 521kB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    blue         (time, y, x) uint16 521kB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    green        (time, y, x) uint16 521kB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    emad         (time, y, x) float32 1MB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    smad         (time, y, x) float32 1MB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    ...           ...
    swir16       (time, y, x) uint16 521kB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    swir22       (time, y, x) uint16 521kB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    coastal      (time, y, x) uint16 521kB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    rededge1     (time, y, x) uint16 521kB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    rededge2     (time, y, x) uint16 521kB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    rededge3     (time, y, x) uint16 521kB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>

In [8]:
scaled_data = scale(data)
indices = calculate_band_indices(scaled_data)

# TODO: split masks into separate functions
masked_data = apply_masks(scaled_data)

masked_data

<xarray.Dataset> Size: 51MB
Dimensions:      (time: 1, y: 269, x: 968)
Coordinates:
  * y            (y) float64 2kB -2.05e+06 -2.05e+06 ... -2.052e+06 -2.052e+06
  * x            (x) float64 8kB 3.086e+06 3.086e+06 ... 3.096e+06 3.096e+06
    spatial_ref  int32 4B 3832
  * time         (time) datetime64[ns] 8B 2024-01-01
Data variables: (12/26)
    nir          (time, y, x) float64 2MB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    red          (time, y, x) float64 2MB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    blue         (time, y, x) float64 2MB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    green        (time, y, x) float64 2MB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    emad         (time, y, x) float32 1MB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    smad         (time, y, x) float32 1MB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    ...           ...
    ndwi         (time, y, x) float64 2MB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    b_g          (time, y, x) float64 2MB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    b_r          (time, y, x) float64 2MB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    mci          (time, y, x) float64 2MB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    ndci         (time, y, x) float64 2MB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>
    ln_bg        (time, y, x) float64 2MB dask.array<chunksize=(1, 269, 968), meta=np.ndarray>

In [9]:
masked_data.odc.explore(bands=["red", "green", "blue"], vmin=0, vmax=0.3)

/srv/conda/envs/notebook/lib/python3.10/site-packages/odc/geo/_rgba.py:55: RuntimeWarning: invalid value encountered in cast
  return x.astype("uint8")
/srv/conda/envs/notebook/lib/python3.10/site-packages/rasterio/warp.py:387: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dest = _reproject(


## Run predictions over our region

In [9]:
model = joblib.load("models/model-geomad-joined-data-rf-11042025.model")

In [10]:
loaded = masked_data.compute()
loaded

In [12]:
predicted_da = do_prediction(loaded, model).astype(np.float32)
predicted_da

<xarray.DataArray (y: 269, x: 968, time: 1)> Size: 1MB
array([[[ 4.],
        [ 1.],
        [ 4.],
        ...,
        [nan],
        [nan],
        [nan]],

       [[ 4.],
        [ 1.],
        [ 4.],
        ...,
        [nan],
        [nan],
        [nan]],

       [[ 4.],
        [ 1.],
        [ 4.],
        ...,
...
        ...,
        [nan],
        [nan],
        [nan]],

       [[ 8.],
        [ 8.],
        [ 8.],
        ...,
        [nan],
        [nan],
        [nan]],

       [[ 8.],
        [ 8.],
        [ 8.],
        ...,
        [nan],
        [nan],
        [nan]]], dtype=float32)
Coordinates:
  * y            (y) float64 2kB -2.05e+06 -2.05e+06 ... -2.052e+06 -2.052e+06
  * x            (x) float64 8kB 3.086e+06 3.086e+06 ... 3.096e+06 3.096e+06
    spatial_ref  int32 4B 3832
  * time         (time) datetime64[ns] 8B 2024-01-01

## Visualisation
Visualise the prediction with the 8 class colour map or the 10 class colour map below, respectively

In [13]:
classes = [
    [1, "sediment", "#8c8c8c"],
    [2, "sand", "#fedd24"],
    [3, "rubble", "#f8ffb4"],
    [4, "seagrass", "#6df7dc"],
    [5, "seaweed", "#b9df6f"],
    [6, "coral", "#a011c3"],
    [7, "rock", "#804600"],
    [8, "deeps", "#011b61"],
]

values_list = [c[0] for c in classes]
color_list = [c[2] for c in classes]

# Build a listed colormap.
c_map = colors.ListedColormap(color_list)
bounds = values_list + [8]
norm = colors.BoundaryNorm(bounds, c_map.N)
predicted_da.odc.explore(cmap=c_map)

In [14]:
classes = [
    [1, "sediment", "#8c8c8c"],
    [2, "sand", "#fedd24"],
    [3, "rubble", "#f8ffb4"],
    [4, "seagrass", "#6df7dc"],
    [5, "seaweed", "#b9df6f"],
    [6, "coral", "#a011c3"],
    [7, "rock", "#804600"],
    [8, "deeps", "#011b61"],
    [9, "mangrove", "#086a39"],
    [10, "land", "#00FFFFFF"],
]

values_list = [c[0] for c in classes]
color_list = [c[2] for c in classes]

# Build a listed colormap.
c_map = colors.ListedColormap(color_list)
bounds = values_list + [14]
norm = colors.BoundaryNorm(bounds, c_map.N)

In [15]:
predicted_da.odc.explore(cmap=c_map)

In [16]:
# predicted_da.odc.write_cog("predictions/predicted_ba_estuary_joined_data_postcard_svm_22042025.tiff", overwrite=True)
predicted_da.odc.write_cog("predictions/predicted_komave_postcard_rf_24042025.tiff", overwrite=True)

PosixPath('predictions/predicted_komave_postcard_rf_24042025.tiff')